In [189]:
import numpy as np
import tensorflow as tf
import pandas as pd

from sklearn.model_selection import train_test_split


from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras.callbacks import TensorBoard
from keras.models import load_model

biden = pd.read_csv("dataset/biden/real/"+"output.csv")
justin = pd.read_csv("dataset/justin/real/"+"output.csv")
#may = pd.read_csv("dataset/may/real/"+"output.csv")
pelosi = pd.read_csv("dataset/pelosi/real/"+"output.csv")
michelle = pd.read_csv("dataset/michelle/real/"+"output.csv")

biden['label'] = 0
justin['label'] = 1
#may['label'] = 'may'
pelosi['label'] = 2
michelle['label'] = 3

df_merged = pd.concat([biden, justin, michelle, pelosi])


Y = df_merged['label']
X = df_merged.drop('label', axis =1)

In [190]:
X_new= X.iloc[0:610250]

In [191]:
from sklearn.preprocessing import StandardScaler
X_new = StandardScaler().fit_transform(X_new)

norm_X = np.array(X_new)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [192]:
timesteps = 50
data_dim = 426
num_classes = 4

In [193]:
updated_X = np.reshape(norm_X, (int(norm_X.shape[0]/timesteps), 50, 426))

In [194]:
updated_X.shape

(12205, 50, 426)

In [197]:
Y = np.array(Y.iloc[0:610250])

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [203]:
i=0
count =0
target = []
for i in range(0,610249, 50):
    z = np.argmax(np.bincount(Y[i:i+50]))
    target.append(z)
    count = count+1

In [204]:
target = np.array(target)

In [206]:
encoder = LabelEncoder()
encoder.fit(target)
encoded_Y = encoder.transform(target)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [207]:
dummy_y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [208]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.layers import LSTM, Dense

In [209]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(426, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 426
model.add(LSTM(200, return_sequences=True))  # returns a sequence of vectors of dimension 200
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(4, activation='softmax'))

In [210]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [211]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_37 (LSTM)               (None, 50, 426)           1453512   
_________________________________________________________________
lstm_38 (LSTM)               (None, 50, 200)           501600    
_________________________________________________________________
lstm_39 (LSTM)               (None, 32)                29824     
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 132       
Total params: 1,985,068
Trainable params: 1,985,068
Non-trainable params: 0
_________________________________________________________________


In [223]:
X_train, X_test, y_train, y_test = train_test_split(updated_X, dummy_y, test_size=0.33, random_state=42)

In [224]:

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
model.fit(X_train, y_train,
          batch_size=64, validation_split=0.3, shuffle = True,  epochs=3, callbacks=[tensorboard])

Train on 5723 samples, validate on 2454 samples
Epoch 1/3
5723/5723 [==============================] - 149s 26ms/step - loss: 0.1462 - acc: 0.9614 - val_loss: 0.0666 - val_acc: 0.9772
Epoch 2/3
5723/5723 [==============================] - 102s 18ms/step - loss: 0.0437 - acc: 0.9892 - val_loss: 0.0160 - val_acc: 0.9955
Epoch 3/3
5723/5723 [==============================] - 102s 18ms/step - loss: 0.0196 - acc: 0.9942 - val_loss: 0.0047 - val_acc: 0.9992


In [225]:
model.save("lstm_facial.h5")
del model

model = load_model("lstm_facial.h5")

In [239]:
y_pred = model.predict_classes(X_test)

In [253]:
X_test.shape

(4028, 50, 426)

In [245]:
len(y_pred)

4028

In [238]:
len(y_test)

4028

In [242]:

#for decoding the one hot encoded input
lst = []
def decode(datum):
    return np.argmax(datum)

for i in range(y_test.shape[0]):
    lst.append(decode(y_test[i]))

In [251]:
accuracy = (y_pred == np.array(lst)).mean()
print(accuracy)

0.9985104270109235


In [221]:
dropped_X = X.drop([X.columns[[1, 69]]], axis=1)

['frame',
 ' face_id',
 ' timestamp',
 ' confidence',
 ' success',
 ' pose_Tx',
 ' pose_Ty',
 ' pose_Tz',
 ' pose_Rx',
 ' pose_Ry',
 ' pose_Rz',
 ' x_0',
 ' x_1',
 ' x_2',
 ' x_3',
 ' x_4',
 ' x_5',
 ' x_6',
 ' x_7',
 ' x_8',
 ' x_9',
 ' x_10',
 ' x_11',
 ' x_12',
 ' x_13',
 ' x_14',
 ' x_15',
 ' x_16',
 ' x_17',
 ' x_18',
 ' x_19',
 ' x_20',
 ' x_21',
 ' x_22',
 ' x_23',
 ' x_24',
 ' x_25',
 ' x_26',
 ' x_27',
 ' x_28',
 ' x_29',
 ' x_30',
 ' x_31',
 ' x_32',
 ' x_33',
 ' x_34',
 ' x_35',
 ' x_36',
 ' x_37',
 ' x_38',
 ' x_39',
 ' x_40',
 ' x_41',
 ' x_42',
 ' x_43',
 ' x_44',
 ' x_45',
 ' x_46',
 ' x_47',
 ' x_48',
 ' x_49',
 ' x_50',
 ' x_51',
 ' x_52',
 ' x_53',
 ' x_54',
 ' x_55',
 ' x_56',
 ' x_57',
 ' x_58',
 ' x_59',
 ' x_60',
 ' x_61',
 ' x_62',
 ' x_63',
 ' x_64',
 ' x_65',
 ' x_66',
 ' x_67',
 ' y_0',
 ' y_1',
 ' y_2',
 ' y_3',
 ' y_4',
 ' y_5',
 ' y_6',
 ' y_7',
 ' y_8',
 ' y_9',
 ' y_10',
 ' y_11',
 ' y_12',
 ' y_13',
 ' y_14',
 ' y_15',
 ' y_16',
 ' y_17',
 ' y_18',
 ' y_